In [4]:
import pandas as pd
pd.options.display.max_columns=200
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import bs4 as bs
import nltk
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize, wordpunct_tokenize, RegexpTokenizer
import re
from nltk.corpus import stopwords
import ast
import tiktoken
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
data = pd.read_csv('data/QueryResults.csv')

In [6]:
#delete balises html with beautiful soup
def clean_html(text):
    soup = bs.BeautifulSoup(text, "html.parser")
    return soup.get_text()

data['Body'] = data['Body'].apply(clean_html)

In [7]:
data = data[['Title', 'Body', 'Tags', 'Id']]

In [8]:
data.head()


,Title,Body,Tags,Id
0,how do I calculate a rolling idxmax,consider the pd.Series s\nimport pandas as pd\...,<python><pandas><numpy><dataframe><series>,40101130
1,Better techniques for trimming leading zeros i...,I've been using this for some time:\nSUBSTRING...,<sql><sql-server><sql-server-2005><tsql><string>,662383
2,"""No X11 DISPLAY variable"" - what does it mean?",I am trying to install a Java application on m...,<java><linux><variables><x11><headless>,662421
3,Object-Oriented Callbacks for C++?,Is there some library that allows me to easily...,<c++><oop><callback><pointer-to-member><eiffel>,3520133
4,Why doesn't .NET find the OpenSSL.NET dll?,"EDIT (the whole question, it was too unclear)\...",<c#><.net><dll><dllimport><dllnotfoundexception>,1396164


In [9]:
import re

# Suppression des chevrons et extraction des tags
data['Tags'] = data['Tags'].apply(lambda x: re.findall(r'<([^>]+)>', x))

data.head()

,Title,Body,Tags,Id
0,how do I calculate a rolling idxmax,consider the pd.Series s\nimport pandas as pd\...,"[python, pandas, numpy, dataframe, series]",40101130
1,Better techniques for trimming leading zeros i...,I've been using this for some time:\nSUBSTRING...,"[sql, sql-server, sql-server-2005, tsql, string]",662383
2,"""No X11 DISPLAY variable"" - what does it mean?",I am trying to install a Java application on m...,"[java, linux, variables, x11, headless]",662421
3,Object-Oriented Callbacks for C++?,Is there some library that allows me to easily...,"[c++, oop, callback, pointer-to-member, eiffel]",3520133
4,Why doesn't .NET find the OpenSSL.NET dll?,"EDIT (the whole question, it was too unclear)\...","[c#, .net, dll, dllimport, dllnotfoundexception]",1396164


In [10]:
data['Tags'].head()

0          [python, pandas, numpy, dataframe, series]
1    [sql, sql-server, sql-server-2005, tsql, string]
2             [java, linux, variables, x11, headless]
3     [c++, oop, callback, pointer-to-member, eiffel]
4    [c#, .net, dll, dllimport, dllnotfoundexception]
Name: Tags, dtype: object

In [11]:
data.head()

,Title,Body,Tags,Id
0,how do I calculate a rolling idxmax,consider the pd.Series s\nimport pandas as pd\...,"[python, pandas, numpy, dataframe, series]",40101130
1,Better techniques for trimming leading zeros i...,I've been using this for some time:\nSUBSTRING...,"[sql, sql-server, sql-server-2005, tsql, string]",662383
2,"""No X11 DISPLAY variable"" - what does it mean?",I am trying to install a Java application on m...,"[java, linux, variables, x11, headless]",662421
3,Object-Oriented Callbacks for C++?,Is there some library that allows me to easily...,"[c++, oop, callback, pointer-to-member, eiffel]",3520133
4,Why doesn't .NET find the OpenSSL.NET dll?,"EDIT (the whole question, it was too unclear)\...","[c#, .net, dll, dllimport, dllnotfoundexception]",1396164


In [12]:
data['Text'] = data['Title'] + ' ' + data['Body']

In [13]:
from collections import Counter

# Compter les tags
tags_counter = Counter()
for tags in data['Tags']:
    tags_counter.update(tags)

# Nombre de tags différents
print("Nombre de tags différents :", len(tags_counter))

# Obtenir les 100 tags les plus fréquents
top_50_tags = tags_counter.most_common(50)
print("\nTop 50 tags les plus fréquents :")
for tag, count in top_50_tags:
    print(f"{tag}: {count}")

Nombre de tags différents : 18315

Top 50 tags les plus fréquents :
c#: 6464
java: 5846
javascript: 4876
python: 4699
c++: 3868
.net: 3488
ios: 3473
android: 3187
html: 2154
php: 2005
objective-c: 1890
jquery: 1785
iphone: 1705
c: 1592
asp.net: 1345
sql: 1337
css: 1303
linux: 1267
node.js: 1142
spring: 1051
performance: 1049
swift: 1026
windows: 1023
ruby-on-rails: 961
xcode: 931
json: 913
mysql: 911
sql-server: 887
multithreading: 843
asp.net-mvc: 805
unit-testing: 787
database: 785
ruby: 781
arrays: 771
django: 770
wpf: 760
macos: 738
visual-studio: 732
c++11: 699
reactjs: 695
algorithm: 692
string: 673
python-3.x: 592
xml: 545
ajax: 535
cocoa-touch: 518
gcc: 508
security: 503
angular: 496
eclipse: 487


In [14]:
top_50_tags_set = set(tag for tag, _ in top_50_tags)

# Fonction pour ne garder que les tags du top 100
def keep_top_50_tags(tags):
    return [tag for tag in tags if tag in top_50_tags_set]

# Appliquer la fonction 
data_filtered = data.copy()
data_filtered['Tags'] = data_filtered['Tags'].apply(keep_top_50_tags)

# Filtrer les données pour ne garder que les questions ayant au moins un tag dans le top 100
data_filtered = data_filtered[data_filtered['Tags'].apply(lambda x: len(x) > 0)].reset_index(drop=True)

# Afficher le nombre de questions restantes
print("Nombre de questions restantes :", len(data_filtered))

Nombre de questions restantes : 43783


In [15]:
import pandas as pd

# Compter le nombre de tokens dans chaque texte
data_filtered['Token_Count'] = data_filtered['Text'].apply(lambda x: len(x.split()))

In [16]:
data_filtered['Token_Count'].describe()

count    43783.000000
mean       189.524176
std        184.684547
min          8.000000
25%         87.000000
50%        140.000000
75%        230.000000
max      10740.000000
Name: Token_Count, dtype: float64

In [17]:
data_filtered.loc[data_filtered['Token_Count'] > 5000, 'Token_Count']

3160    10740
Name: Token_Count, dtype: int64

In [18]:
data_filtered.drop(data_filtered.loc[data_filtered['Token_Count'] > 5000].index, inplace=True)
data_filtered.drop(data_filtered.loc[data_filtered['Body'].str.len() > 10000].index, inplace=True)

In [19]:
#Split to get 5000 questions
data_filtered = data_filtered[:5000]

In [20]:
data_filtered.reset_index(drop=True, inplace=True)

In [21]:
data_filtered

,Title,Body,Tags,Id,Text,Token_Count
0,how do I calculate a rolling idxmax,consider the pd.Series s\nimport pandas as pd\...,[python],40101130,how do I calculate a rolling idxmax consider t...,147
1,Better techniques for trimming leading zeros i...,I've been using this for some time:\nSUBSTRING...,"[sql, sql-server, string]",662383,Better techniques for trimming leading zeros i...,103
2,"""No X11 DISPLAY variable"" - what does it mean?",I am trying to install a Java application on m...,"[java, linux]",662421,"""No X11 DISPLAY variable"" - what does it mean?...",166
3,Object-Oriented Callbacks for C++?,Is there some library that allows me to easily...,[c++],3520133,Object-Oriented Callbacks for C++? Is there so...,206
4,Why doesn't .NET find the OpenSSL.NET dll?,"EDIT (the whole question, it was too unclear)\...","[c#, .net]",1396164,Why doesn't .NET find the OpenSSL.NET dll? EDI...,151
...,...,...,...,...,...,...
4995,Visual studio 2013 + .Net 4.5.1 + Edit and con...,Supposedly vs 2013 added support for edit and ...,"[.net, visual-studio]",20324080,Visual studio 2013 + .Net 4.5.1 + Edit and con...,195
4996,How to set a default parameter for a vector <s...,"For example, a class named Table, with its con...","[c++, string]",1854241,How to set a default parameter for a vector <s...,50
4997,Fast sine/cosine for ARMv7+NEON: looking for t...,Could somebody with access to an iPhone 3GS or...,[performance],1854254,Fast sine/cosine for ARMv7+NEON: looking for t...,264
4998,Quicksort superiority over Heap Sort,Heap Sort has a worst case complexity of O(nlo...,[algorithm],1853208,Quicksort superiority over Heap Sort Heap Sort...,28


In [22]:
data_filtered.shape

(5000, 6)

In [23]:
def tokenizer_fct(sentence) :
    # print(sentence)
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    word_tokens = word_tokenize(sentence_clean)
    return word_tokens

# Stop words
stop_w = list(set(stopwords.words('english'))) + ['[', ']', ',', '.', ':', '?', '(', ')', "'", '"', '!', ';', '``', "''", '...', '’', '“', '”']

def stop_word_filter_fct(list_words) :
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

# lower case et alpha
def lower_start_fct(list_words) :
    lw = [w.lower() for w in list_words if (not w.startswith("@")) 
                                       and (not w.startswith("#"))
                                       and (not w.startswith("http"))]
    return lw

# Lemmatizer (base d'un mot)
def lemma_fct(list_words) :
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

# Fonction de préparation du texte pour le bag of words avec lemmatization
def transform_bow_lem_fct(desc_text) :
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    lem_w = lemma_fct(lw)    
    transf_desc_text = ' '.join(lem_w)
    return transf_desc_text

# Fonction de préparation du texte pour le Deep learning (USE et BERT)
def transform_dl_fct(desc_text) :
    word_tokens = tokenizer_fct(desc_text)
    #sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(word_tokens)
    # lem_w = lemma_fct(lw)    
    transf_desc_text = ' '.join(lw)
    return transf_desc_text

def transform_tiktoken(desc_text):
    # Utilisation de tiktoken pour l'encodage
    enc = tiktoken.get_encoding("cl100k_base")
    word_tokens = enc.encode(desc_text)
    decoded_text = enc.decode(word_tokens)
    # Conversion en minuscules (si nécessaire)
    #decoded_text = decoded_text.lower()
        
    return decoded_text

In [24]:
X = data_filtered['Title'].apply(transform_bow_lem_fct)
data_bow = pd.DataFrame(X)
data_bow['Body'] = data_filtered['Body'].apply(transform_bow_lem_fct)
data_bow['Text'] = data_filtered['Text'].apply(transform_bow_lem_fct)
data_bow['Tags'] = data_filtered['Tags']

In [25]:
Y = data_filtered['Title'].apply(transform_tiktoken)
data_se = pd.DataFrame(Y)
data_se['Body'] = data_filtered['Body'].apply(transform_tiktoken)
data_se['Text'] = data_filtered['Text'].apply(transform_tiktoken)
data_se['Tags'] = data_filtered['Tags']
data_se['Token_Count'] = data_filtered['Token_Count']

In [26]:
data_bow.head()

,Title,Body,Text,Tags
0,calculate rolling idxmax,consider pd.series import panda import numpy n...,calculate rolling idxmax consider pd.series im...,[python]
1,better technique trimming leading zero sql server,'ve using time substring str_col patindex str_...,better technique trimming leading zero sql ser...,"[sql, sql-server, string]"
2,x11 display variable mean,trying install java application linux machine ...,x11 display variable mean trying install java ...,"[java, linux]"
3,object oriented callback,library allows easily conveniently create obje...,object oriented callback library allows easily...,[c++]
4,why n't .net find openssl.net dll,edit whole question unclear want use openssl.n...,why n't .net find openssl.net dll edit whole q...,"[c#, .net]"


In [27]:
# Split train test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_bow[['Title','Text']], data_bow['Tags'], test_size=0.2, random_state=0)
X_train_se, X_test_se, y_train_se, y_test_se = train_test_split(data_se[['Text','Body']], data_se['Tags'], test_size=0.2, random_state=0)

In [28]:
X_train_se

,Text,Body
2913,Using Visual Studio Code tasks to automate C m...,I have a project written in C that has two mak...
3275,How to automatically append text to text copie...,"In JavaScript, how can you select text on a we..."
775,Get list of apps of all users If I want to ret...,If I want to retrieve an ApplicationInfo list ...
217,"If two languages follow IEEE 754, will calcula...",I'm in the process of converting a program fro...
1245,SQLAlchemy: How to make an integer column auto...,I am using Flask extension for SQLAlchemy to d...
...,...,...
4931,"WPF: How to detect Key repetition, in Key* eve...",NOTE: e.IsRepeat is confirmed to work. The pro...
3264,What is the most compatible way to install pyt...,I'm starting to learn python and loving it. I ...
1653,JUnit5: How to assert several properties of an...,I want to assert several properties of an obje...
2607,Browser Caching in ASP.NET application Any sug...,Any suggestions on how to do browser caching w...


In [29]:
X_train

,Title,Text
2913,using visual studio code task automate makefil...,using visual studio code task automate makefil...
3275,how automatically append text text copied java...,how automatically append text text copied java...
775,get list apps user,get list apps user want retrieve applicationin...
217,two language follow ieee 754 calculation langu...,two language follow ieee 754 calculation langu...
1245,sqlalchemy how make integer column auto_increm...,sqlalchemy how make integer column auto_increm...
...,...,...
4931,wpf how detect key repetition key event,wpf how detect key repetition key event note e...
3264,what compatible way install python module mac,what compatible way install python module mac ...
1653,junit5 how assert several property object sing...,junit5 how assert several property object sing...
2607,browser caching asp.net application,browser caching asp.net application any sugges...


In [30]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_train_encoded = mlb.fit_transform(y_train)
y_test_encoded = mlb.transform(y_test)

In [31]:
mlb.classes_.shape

(50,)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

ctf = TfidfVectorizer(stop_words='english', max_df=0.9, min_df=2)
ctf_fit = ctf.fit(X_train['Title'])
ctf_text_train = ctf.transform(X_train['Text'])
ctf_text_test = ctf.transform(X_test['Text'])

In [33]:
len(ctf_fit.vocabulary_)

2267

In [34]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [35]:
ovr = OneVsRestClassifier(LogisticRegression(), n_jobs=-3)
ovr.fit(ctf_text_train, y_train_encoded)
y_pred_test_ctf = ovr.predict(ctf_text_test)

In [36]:
from sklearn.metrics import jaccard_score

In [37]:
score_ctf = jaccard_score(y_test_encoded, y_pred_test_ctf, average='weighted')

print("Jaccard Score ctf:", score_ctf)

Jaccard Score ctf: 0.14732754533162903


In [38]:
"""from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline

# Créer un classificateur OvR avec une régression logistique
ovr = OneVsRestClassifier(LogisticRegression())

# Créer un pipeline avec TfidfVectorizer et OneVsRestClassifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', ovr)
])

# Définir les paramètres de la grille à optimiser
param_grid = [
    {
        'tfidf__max_df': [0.9, 1.0],
        'tfidf__min_df': [1, 2],
        'tfidf__stop_words': [None, 'english'],
        'clf__estimator__C': [0.1, 1, 10],
        'clf__estimator__penalty': ['l1'],
        'clf__estimator__solver': ['liblinear', 'saga'],
    },
    {
        'tfidf__max_df': [0.9, 1.0],
        'tfidf__min_df': [1, 2],
        'tfidf__stop_words': [None, 'english'],
        'clf__estimator__C': [0.1, 1, 10],
        'clf__estimator__penalty': ['l2'],
        'clf__estimator__solver': ['newton-cg', 'lbfgs', 'sag', 'saga', 'liblinear'],
    },
]

# Utiliser le score de Jaccard pondéré pour l'évaluation
jaccard_scorer = make_scorer(jaccard_score, average='weighted')

# Créer l'objet GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, scoring=jaccard_scorer, cv=5, n_jobs=-3)

# Ajuster la recherche par grille aux données
grid_search.fit(X_train['Text'], y_train_encoded)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres trouvés:")
print(grid_search.best_params_)"""

'from sklearn.model_selection import GridSearchCV\nfrom sklearn.metrics import make_scorer\nfrom sklearn.pipeline import Pipeline\n\n# Créer un classificateur OvR avec une régression logistique\novr = OneVsRestClassifier(LogisticRegression())\n\n# Créer un pipeline avec TfidfVectorizer et OneVsRestClassifier\npipeline = Pipeline([\n    (\'tfidf\', TfidfVectorizer()),\n    (\'clf\', ovr)\n])\n\n# Définir les paramètres de la grille à optimiser\nparam_grid = [\n    {\n        \'tfidf__max_df\': [0.9, 1.0],\n        \'tfidf__min_df\': [1, 2],\n        \'tfidf__stop_words\': [None, \'english\'],\n        \'clf__estimator__C\': [0.1, 1, 10],\n        \'clf__estimator__penalty\': [\'l1\'],\n        \'clf__estimator__solver\': [\'liblinear\', \'saga\'],\n    },\n    {\n        \'tfidf__max_df\': [0.9, 1.0],\n        \'tfidf__min_df\': [1, 2],\n        \'tfidf__stop_words\': [None, \'english\'],\n        \'clf__estimator__C\': [0.1, 1, 10],\n        \'clf__estimator__penalty\': [\'l2\'],\n    

best_params = {'clf__estimator__C': 10, 'clf__estimator__penalty': 'l1', 'clf__estimator__solver': 'liblinear', 'tfidf__max_df': 0.9, 'tfidf__min_df': 1, 'tfidf__stop_words': None}

In [39]:
# Appliquer les meilleurs paramètres pour TfidfVectorizer
tfidf = TfidfVectorizer(max_df=1.0, min_df=1, stop_words=None)

# Transformer les données d'entraînement et de test
X_train_tfidf = tfidf.fit_transform(X_train['Text'])
X_test_tfidf = tfidf.transform(X_test['Text'])

In [40]:
# Créer le classificateur OvR en utilisant les meilleurs paramètres
classic = OneVsRestClassifier(LogisticRegression(C=10, penalty='l1', solver='liblinear'))

# Entraîner le classificateur avec les données d'entraînement transformées
classic.fit(X_train_tfidf, y_train_encoded)

OneVsRestClassifier(estimator=LogisticRegression(C=10, penalty='l1',
                                                 solver='liblinear'))

In [41]:
jaccard_score(y_test_encoded, classic.predict(X_test_tfidf), average='weighted')

0.4225233809166425

In [42]:
import random
from termcolor import colored


# Utiliser predict_proba pour obtenir les probabilités des tags
y_pred_proba = classic.predict_proba(X_test_tfidf)

# Obtenir les vrais tags pour les données de test
y_test_tags = mlb.inverse_transform(y_test_encoded)

# Appliquer un seuil pour les probabilités
threshold = 0.1

# Afficher les prédictions et les vrais tags pour 10 exemples pris au hasard
num_examples = 5
random_examples = random.sample(range(len(X_test)), num_examples)

for i in random_examples:
    print(f"Exemple {i + 1}:")
    print(f"Texte  : {X_test.iloc[i]['Text']}")
    print(f"Vrais tags  : {' | '.join(y_test_tags[i])}")

    # Extraire les tags prédits au-dessus du seuil avec leurs probabilités
    predicted_tags_with_probs = [(mlb.classes_[j], prob) for j, prob in enumerate(y_pred_proba[i]) if prob > threshold]
    
    # Ajouter les tags réels qui ne sont pas déjà dans les tags prédits
    for tag in y_test_tags[i]:
        if tag not in [t[0] for t in predicted_tags_with_probs]:
            prob = y_pred_proba[i][mlb.classes_.tolist().index(tag)]
            predicted_tags_with_probs.append((tag, prob))

    # Trier les tags prédits et réels par popularité et les afficher avec leurs probabilités
    sorted_tags_with_probs = sorted(predicted_tags_with_probs, key=lambda x: x[1], reverse=True)
    
    colored_tags = []
    for tag, prob in sorted_tags_with_probs:
        if tag in y_test_tags[i] and prob > threshold:
            colored_tags.append(colored(f"{tag} ({prob:.3f})", 'green'))
        elif tag in y_test_tags[i]:
            colored_tags.append(colored(f"{tag} ({prob:.3f})", 'red'))
        else:
            colored_tags.append(colored(f"{tag} ({prob:.3f})", 'yellow'))
    
    sorted_tags_with_probs_str = ' | '.join(colored_tags)
    
    print(f"Tags prédits et réels  : {sorted_tags_with_probs_str}")
    print("-" * 50)

Exemple 479:
Texte  : optimized implementation java.util.map java.util.set writing application memory lesser extent speed vital found profiling spend great deal time map set operation while look way call method le wondering whether anyone written come across implementation significantly improve access time memory overhead least improve thing given assumption from looking jdk source n't believe n't made faster leaner aware common collection n't believe implementation whose goal faster leaner same google collection update should noted need thread safety
Vrais tags  : java | performance
Tags prédits et réels  : java (0.999) | performance (0.847) | c (0.106)
--------------------------------------------------
Exemple 900:
Texte  : layered database application without using orm specify need data display looking pointer information 'll make since suspect single one correct answer this hence 'll make reference linq also apologize long post let summarize question full question follows summary b

Partie 2 : GPT-4

In [43]:
import openai
from openai.embeddings_utils import get_embedding

In [44]:
openai.api_key=input("Enter your API key: ")

In [45]:
'''X_train_se['Embedding_body'] = None
embedding_model = "text-embedding-ada-002"
for i in range(len(X_train_se)):
    embedding = get_embedding(X_train_se.iloc[i]["Body"], engine=embedding_model)
    X_train_se.iat[i, X_train_se.columns.get_loc("Embedding_body")] = embedding'''

'X_train_se[\'Embedding_body\'] = None\nembedding_model = "text-embedding-ada-002"\nfor i in range(len(X_train_se)):\n    embedding = get_embedding(X_train_se.iloc[i]["Body"], engine=embedding_model)\n    X_train_se.iat[i, X_train_se.columns.get_loc("Embedding_body")] = embedding'

In [46]:
'''X_test_se['Embedding_body'] = None
embedding_model = "text-embedding-ada-002"
for i in range(len(X_test_se)):
    embedding = get_embedding(X_test_se.iloc[i]["Body"], engine=embedding_model)
    X_test_se.iat[i, X_test_se.columns.get_loc("Embedding_body")] = embedding'''

'X_test_se[\'Embedding_body\'] = None\nembedding_model = "text-embedding-ada-002"\nfor i in range(len(X_test_se)):\n    embedding = get_embedding(X_test_se.iloc[i]["Body"], engine=embedding_model)\n    X_test_se.iat[i, X_test_se.columns.get_loc("Embedding_body")] = embedding'

In [47]:
"""X_train_se['Embedding_text'] = None
embedding_model = "text-embedding-ada-002"
for i in range(len(X_train_se)):
      embedding = get_embedding(X_train_se.iloc[i]["Text"], engine=embedding_model)
      X_train_se.iat[i, X_train_se.columns.get_loc("Embedding_text")] = embedding"""

'X_train_se[\'Embedding_text\'] = None\nembedding_model = "text-embedding-ada-002"\nfor i in range(len(X_train_se)):\n      embedding = get_embedding(X_train_se.iloc[i]["Text"], engine=embedding_model)\n      X_train_se.iat[i, X_train_se.columns.get_loc("Embedding_text")] = embedding'

In [48]:
"""X_test_se['Embedding_text'] = None
embedding_model = "text-embedding-ada-002"
for i in range(len(X_test_se)):
      embedding = get_embedding(X_test_se.iloc[i]["Text"], engine=embedding_model)
      X_test_se.iat[i, X_test_se.columns.get_loc("Embedding_text")] = embedding"""

'X_test_se[\'Embedding_text\'] = None\nembedding_model = "text-embedding-ada-002"\nfor i in range(len(X_test_se)):\n      embedding = get_embedding(X_test_se.iloc[i]["Text"], engine=embedding_model)\n      X_test_se.iat[i, X_test_se.columns.get_loc("Embedding_text")] = embedding'

In [49]:
"""#Save X_test_se and X_train_se
X_test_se.to_csv("lfs_data/X_test_se.csv", index=False)
X_train_se.to_csv("lfs_data/X_train_se.csv", index=False)"""

'#Save X_test_se and X_train_se\nX_test_se.to_csv("lfs_data/X_test_se.csv", index=False)\nX_train_se.to_csv("lfs_data/X_train_se.csv", index=False)'

In [50]:
#Load X_test_se and X_train_se
X_test_se = pd.read_csv("lfs_data/X_test_se.csv")
X_train_se = pd.read_csv("lfs_data/X_train_se.csv")

In [51]:
X_train_se

,Text,Body,Embedding_text
0,Using Visual Studio Code tasks to automate C m...,I have a project written in C that has two mak...,"[-0.030831696465611458, 0.006386663764715195, ..."
1,How to automatically append text to text copie...,"In JavaScript, how can you select text on a we...","[-0.010805660858750343, 0.00861301738768816, 0..."
2,Get list of apps of all users If I want to ret...,If I want to retrieve an ApplicationInfo list ...,"[0.0025181155651807785, 0.008718851022422314, ..."
3,"If two languages follow IEEE 754, will calcula...",I'm in the process of converting a program fro...,"[0.0032508610747754574, -0.004044834524393082,..."
4,SQLAlchemy: How to make an integer column auto...,I am using Flask extension for SQLAlchemy to d...,"[-0.011885248124599457, 0.023266715928912163, ..."
...,...,...,...
3995,"WPF: How to detect Key repetition, in Key* eve...",NOTE: e.IsRepeat is confirmed to work. The pro...,"[-0.013747887685894966, -0.0019125896506011486..."
3996,What is the most compatible way to install pyt...,I'm starting to learn python and loving it. I ...,"[0.004587247967720032, -0.016147112473845482, ..."
3997,JUnit5: How to assert several properties of an...,I want to assert several properties of an obje...,"[-0.013546628877520561, 0.02201327122747898, -..."
3998,Browser Caching in ASP.NET application Any sug...,Any suggestions on how to do browser caching w...,"[-0.014035265892744064, 0.029645787551999092, ..."


In [52]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_train_se = mlb.fit_transform(y_train_se)
y_test_se = mlb.transform(y_test_se)

In [53]:
import numpy as np
import ast

def convert_embedding_string_to_array(embedding_string):
    # Convertir la chaîne en liste
    embedding_list = ast.literal_eval(embedding_string)
    # Convertir la liste en tableau numpy
    return np.array(embedding_list)

In [54]:
"""# Convertir les embeddings_body de chaînes de caractères en tableaux numpy
X_train_se['Embedding_body'] = X_train_se['Embedding_body'].apply(convert_embedding_string_to_array)
X_test_se['Embedding_body'] = X_test_se['Embedding_body'].apply(convert_embedding_string_to_array)

# Empilez les embeddings dans des tableaux numpy pour l'entraînement
X_train_embeddings_body = np.stack(X_train_se['Embedding_body'].values)
X_test_embeddings_body = np.stack(X_test_se['Embedding_body'].values)"""

"# Convertir les embeddings_body de chaînes de caractères en tableaux numpy\nX_train_se['Embedding_body'] = X_train_se['Embedding_body'].apply(convert_embedding_string_to_array)\nX_test_se['Embedding_body'] = X_test_se['Embedding_body'].apply(convert_embedding_string_to_array)\n\n# Empilez les embeddings dans des tableaux numpy pour l'entraînement\nX_train_embeddings_body = np.stack(X_train_se['Embedding_body'].values)\nX_test_embeddings_body = np.stack(X_test_se['Embedding_body'].values)"

In [55]:
# Convertir les embeddings_text de chaînes de caractères en tableaux numpy
X_train_se['Embedding_text'] = X_train_se['Embedding_text'].apply(convert_embedding_string_to_array)
X_test_se['Embedding_text'] = X_test_se['Embedding_text'].apply(convert_embedding_string_to_array)

# Empilez les embeddings dans des tableaux numpy pour l'entraînement
X_train_embeddings_text = np.stack(X_train_se['Embedding_text'].values)
X_test_embeddings_text = np.stack(X_test_se['Embedding_text'].values)

In [56]:
"""# Sélectionner un modèle
model = OneVsRestClassifier(LogisticRegression(C=10, penalty='l1', solver='liblinear'), n_jobs=-3)

# Entraîner le modèle
model.fit(X_train_embeddings_body, y_train_se)"""

"# Sélectionner un modèle\nmodel = OneVsRestClassifier(LogisticRegression(C=10, penalty='l1', solver='liblinear'), n_jobs=-3)\n\n# Entraîner le modèle\nmodel.fit(X_train_embeddings_body, y_train_se)"

In [57]:
"""#save model
import pickle
pickle.dump(model, open("lfs_data/ovr_ada_body.pkl", "wb"))"""

'#save model\nimport pickle\npickle.dump(model, open("lfs_data/ovr_ada_body.pkl", "wb"))'

In [58]:
"""#load model
import pickle
ada_body = pickle.load(open("lfs_data/ovr_ada_body.pkl", "rb"))

# Faire des prédictions sur l'ensemble de test
y_pred_ada_body = ada_body.predict(X_test_embeddings_body)"""

'#load model\nimport pickle\nada_body = pickle.load(open("lfs_data/ovr_ada_body.pkl", "rb"))\n\n# Faire des prédictions sur l\'ensemble de test\ny_pred_ada_body = ada_body.predict(X_test_embeddings_body)'

In [59]:
"""# Évaluer le modèle
jaccard_score(y_test_se, y_pred_ada_body, average='weighted')"""

"# Évaluer le modèle\njaccard_score(y_test_se, y_pred_ada_body, average='weighted')"

In [60]:
import numpy as np

# Sélectionner un modèle
ada_txt = OneVsRestClassifier(LogisticRegression(C=10, penalty='l1', solver='liblinear'), n_jobs=-3)

# Entraîner le modèle
ada_txt.fit(X_train_embeddings_text, y_train_se)

OneVsRestClassifier(estimator=LogisticRegression(C=10, penalty='l1',
                                                 solver='liblinear'),
                    n_jobs=-3)

In [61]:
#Save model
import pickle
pickle.dump(ada_txt, open("lfs_data/ovr_ada_text.pkl", "wb"))

In [62]:
#load model
import pickle
ada_text = pickle.load(open("lfs_data/ovr_ada_text.pkl", "rb"))

In [63]:
# Faire des prédictions sur l'ensemble de test
y_pred_ada_text = ada_text.predict(X_test_embeddings_text)

In [64]:
# Évaluer le modèle
jaccard_score(y_test_se, y_pred_ada_text, average='weighted')

0.5302968398584846

In [65]:
import random
from termcolor import colored


# Utiliser predict_proba pour obtenir les probabilités des tags
y_pred_proba = ada_text.predict_proba(X_test_embeddings_text)

# Obtenir les vrais tags pour les données de test
y_test_tags = mlb.inverse_transform(y_test_se)

# Appliquer un seuil pour les probabilités
threshold = 0.1

for i in random_examples:
    print(f"Exemple {i + 1}:")
    print(f"Texte  : {X_test.iloc[i]['Text']}")
    print(f"Vrais tags  : {' | '.join(map(str, y_test_tags[i]))}")

    # Extraire les tags prédits au-dessus du seuil avec leurs probabilités
    predicted_tags_with_probs = [(mlb.classes_[j], prob) for j, prob in enumerate(y_pred_proba[i]) if prob > threshold]
    
    # Ajouter les tags réels qui ne sont pas déjà dans les tags prédits
    for tag in y_test_tags[i]:
        if tag not in [t[0] for t in predicted_tags_with_probs]:
            prob = y_pred_proba[i][mlb.classes_.tolist().index(tag)]
            predicted_tags_with_probs.append((tag, prob))

    # Trier les tags prédits et réels par popularité et les afficher avec leurs probabilités
    sorted_tags_with_probs = sorted(predicted_tags_with_probs, key=lambda x: x[1], reverse=True)
    
    colored_tags = []
    for tag, prob in sorted_tags_with_probs:
        if tag in y_test_tags[i] and prob > threshold:
            colored_tags.append(colored(f"{tag} ({prob:.3f})", 'green'))
        elif tag in y_test_tags[i]:
            colored_tags.append(colored(f"{tag} ({prob:.3f})", 'red'))
        else:
            colored_tags.append(colored(f"{tag} ({prob:.3f})", 'yellow'))
    
    sorted_tags_with_probs_str = ' | '.join(colored_tags)
    
    print(f"Tags prédits et réels  : {sorted_tags_with_probs_str}")
    print("-" * 50)

Exemple 479:
Texte  : optimized implementation java.util.map java.util.set writing application memory lesser extent speed vital found profiling spend great deal time map set operation while look way call method le wondering whether anyone written come across implementation significantly improve access time memory overhead least improve thing given assumption from looking jdk source n't believe n't made faster leaner aware common collection n't believe implementation whose goal faster leaner same google collection update should noted need thread safety
Vrais tags  : java | performance
Tags prédits et réels  : java (0.993) | performance (0.900) | algorithm (0.186) | c# (0.181)
--------------------------------------------------
Exemple 900:
Texte  : layered database application without using orm specify need data display looking pointer information 'll make since suspect single one correct answer this hence 'll make reference linq also apologize long post let summarize question full quest

In [66]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score

# Créer une instance de DummyClassifier
dummy = DummyClassifier(strategy="stratified", random_state=42)

# Adapter le modèle aux données d'entraînement
dummy.fit(X_train, y_train_se) # Utiliser y_train_se directement

# Prédire les tags pour les données de test
y_pred_dummy = dummy.predict(X_test)

In [67]:
jaccard_score(y_test_se, y_pred_dummy, average='weighted')

0.03599652111222306

In [71]:
from termcolor import colored

# Obtenir les probabilités des tags pour ada_text
y_pred_proba_ada_text = ada_text.predict_proba(X_test_embeddings_text)

# Obtenir les probabilités des tags pour classic
y_pred_proba_classic = classic.predict_proba(X_test_tfidf)

# Obtenir les tags prédits pour dummy
y_pred_dummy = dummy.predict(X_test)

# Obtenir les vrais tags pour les données de test
y_test_tags = mlb.inverse_transform(y_test_se)

# Appliquer un seuil pour les probabilités
threshold = 0.1

for i in random_examples:
    print(f"Exemple {i + 1}:")
    print(f"Titre de la question  : {X_test.iloc[i]['Title']}")
    print(f"Vrais tags  : {' | '.join(map(str, y_test_tags[i]))}")

    # Tags prédits par dummy
    dummy_tags = mlb.inverse_transform(y_pred_dummy[i].reshape(1, -1))[0]
    dummy_tags_colored = []
    for tag in dummy_tags:
        color = 'green' if tag in y_test_tags[i] else 'yellow'
        dummy_tags_colored.append(colored(tag, color))
    print(f"Tags prédits par dummy: {' | '.join(dummy_tags_colored)}")

    # Tags prédits par classic
    predicted_tags_classic = [(mlb.classes_[j], prob) for j, prob in enumerate(y_pred_proba_classic[i]) if prob > threshold]
    for tag in y_test_tags[i]:
        if tag not in [t[0] for t in predicted_tags_classic]:
            prob = y_pred_proba_classic[i][mlb.classes_.tolist().index(tag)]
            predicted_tags_classic.append((tag, prob))
    sorted_tags_classic = sorted(predicted_tags_classic, key=lambda x: x[1], reverse=True)
    colored_tags_classic = []
    for tag, prob in sorted_tags_classic:
        if tag in y_test_tags[i] and prob > threshold:
            colored_tags_classic.append(colored(f"{tag} ({prob:.3f})", 'green'))
        elif tag in y_test_tags[i]:
            colored_tags_classic.append(colored(f"{tag} ({prob:.3f})", 'red'))
        else:
            colored_tags_classic.append(colored(f"{tag} ({prob:.3f})", 'yellow'))
    print(f"Tags prédits par classic: {' | '.join(colored_tags_classic)}")

    # Tags prédits par ada_text
    predicted_tags_ada_text = [(mlb.classes_[j], prob) for j, prob in enumerate(y_pred_proba_ada_text[i]) if prob > threshold]
    for tag in y_test_tags[i]:
        if tag not in [t[0] for t in predicted_tags_ada_text]:
            prob = y_pred_proba_ada_text[i][mlb.classes_.tolist().index(tag)]
            predicted_tags_ada_text.append((tag, prob))
    sorted_tags_ada_text = sorted(predicted_tags_ada_text, key=lambda x: x[1], reverse=True)
    colored_tags_ada_text = []
    for tag, prob in sorted_tags_ada_text:
        if tag in y_test_tags[i] and prob > threshold:
            colored_tags_ada_text.append(colored(f"{tag} ({prob:.3f})", 'green'))
        elif tag in y_test_tags[i]:
            colored_tags_ada_text.append(colored(f"{tag} ({prob:.3f})", 'red'))
        else:
            colored_tags_ada_text.append(colored(f"{tag} ({prob:.3f})", 'yellow'))
    print(f"Tags prédits par ada_text: {' | '.join(colored_tags_ada_text)}")

    print("-" * 50)


Exemple 479:
Titre de la question  : optimized implementation java.util.map java.util.set
Vrais tags  : java | performance
Tags prédits par dummy: .net | ruby
Tags prédits par classic: java (0.999) | performance (0.847) | c (0.106)
Tags prédits par ada_text: java (0.993) | performance (0.900) | algorithm (0.186) | c# (0.181)
--------------------------------------------------
Exemple 900:
Titre de la question  : layered database application without using orm specify need data display
Vrais tags  : c#
Tags prédits par dummy: jquery
Tags prédits par classic: asp.net (0.653) | sql (0.458) | c# (0.450) | database (0.160)
Tags prédits par ada_text: c# (0.928) | .net (0.576) | database (0.153)
--------------------------------------------------
Exemple 898:
Titre de la question  : angular get object array
Vrais tags  : angular | arrays | json
Tags prédits par dummy: .net | php | sql
Tags prédits par classic: .net (0.267) | c# (0.238) | performance (0.126) | arrays (0.003) | angular (0.001) | j